# Pacientes similares 

In [1]:
import pandas as pd

In [6]:
sintomas = [
    "fiebre", "pustulas", "distribucion_pustulas", "sobre_areas_enrojecidas", 
    "otros_sintomas", "dolor_articulaciones", "lesiones_escamosas", "psoriasis",
    "familiares_psoriasis", "brotes_similares", "lesion_mas_de_3_meses", 
    "sintomas_post_infeccion", "cambio_medicamento", "estres_infecciones", 
    "vacunacion_reciente"
]

In [65]:
df = pd.read_csv("../db/cuestionario_samples_50.csv")
df.head()

,ehr,fiebre,pustulas,distribucion_pustulas,sobre_areas_enrojecidas,otros_sintomas,dolor_articulaciones,lesiones_escamosas,psoriasis,familiares_psoriasis,brotes_similares,lesion_mas_de_3_meses,sintomas_post_infeccion,cambio_medicamento,estres_infecciones,vacunacion_reciente,fecha,resultado
0,0,Sí,Sí,No,No,No,No,Sí,No,No,No,Sí,No,No,Sí,No,2023-03-16 22:25:48,Sospecha
1,1,Sí,No,No,No sabe,Sí,Sí,No,Sí,No,No,Sí,Sí,Sí,No,No,2024-03-10 13:17:14,Sospecha
2,2,No,Sí,No,No sabe,No,Sí,No,Sí,Sí,No,No,Sí,Sí,No,Sí,2023-01-11 10:41:45,Sospecha
3,3,Sí,Sí,No sabe,No sabe,Sí,No,No,Sí,Sí,Sí,No,No,No,No,No,2023-04-26 10:16:23,Sospecha
4,4,Sí,No,No,No,No,Sí,Sí,Sí,No,No,Sí,Sí,Sí,Sí,Sí,2020-06-25 15:25:10,Sospecha


In [63]:
df.columns

Index(['ehr', 'fiebre', 'pustulas', 'distribucion_pustulas',
       'sobre_areas_enrojecidas', 'otros_sintomas', 'dolor_articulaciones',
       'lesiones_escamosas', 'psoriasis', 'familiares_psoriasis',
       'brotes_similares', 'lesion_mas_de_3_meses', 'sintomas_post_infeccion',
       'cambio_medicamento', 'estres_infecciones', 'vacunacion_reciente',
       'fecha', 'resultado'],
      dtype='object')

In [56]:
from sklearn.cluster import KMeans

def preprocesing(df):
    df_preprocess = df.copy()

    df_preprocess = pd.get_dummies(df_preprocess, columns=['resultado'], prefix=['resultado'], dtype=int)

    df_preprocess = df_preprocess.replace({'Sí': 1, 'No': 0, 'No sabe': 0})

   
    return df_preprocess

def training(df):
    df_clasify = df.copy()

    min_neighbors = 5
    n_clusters = len(df_clasify) // min_neighbors 
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    df_clasify['cluster'] = kmeans.fit_predict(df_clasify.select_dtypes(include=['number']))
    
    return df_clasify


def get_patients_from_same_cluster(ehr_id: int, df: pd.DataFrame) -> pd.DataFrame:
    patient_cluster = df[df['ehr'] == ehr_id]['cluster'].iloc[0]
    
    same_cluster_patients = df[df['cluster'] == patient_cluster]
    
    return same_cluster_patients

In [57]:
df_preprocess = preprocesing(df)
df_preprocess.head()

,ehr,fiebre,pustulas,distribucion_pustulas,sobre_areas_enrojecidas,otros_sintomas,dolor_articulaciones,lesiones_escamosas,psoriasis,familiares_psoriasis,brotes_similares,lesion_mas_de_3_meses,sintomas_post_infeccion,cambio_medicamento,estres_infecciones,vacunacion_reciente,fecha,resultado_No,resultado_Probablemente,resultado_Sospecha
0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,1,0,2023-03-16 22:25:48,0,0,1
1,1,1,0,0,0,1,1,0,1,0,0,1,1,1,0,0,2024-03-10 13:17:14,0,0,1
2,2,0,1,0,0,0,1,0,1,1,0,0,1,1,0,1,2023-01-11 10:41:45,0,0,1
3,3,1,1,0,0,1,0,0,1,1,1,0,0,0,0,0,2023-04-26 10:16:23,0,0,1
4,4,1,0,0,0,0,1,1,1,0,0,1,1,1,1,1,2020-06-25 15:25:10,0,0,1


In [43]:
df_preprocess.describe()

,ehr,fiebre,pustulas,distribucion_pustulas,sobre_areas_enrojecidas,otros_sintomas,dolor_articulaciones,lesiones_escamosas,psoriasis,familiares_psoriasis,brotes_similares,lesion_mas_de_3_meses,sintomas_post_infeccion,cambio_medicamento,estres_infecciones,vacunacion_reciente,resultado_No,resultado_Probablemente,resultado_Sospecha
count,50.00000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,24.50000,0.500000,0.460000,0.160000,0.100000,0.480000,0.480000,0.460000,0.480000,0.440000,0.500000,0.540000,0.520000,0.380000,0.460000,0.480000,0.180000,0.080000,0.740000
std,14.57738,0.505076,0.503457,0.370328,0.303046,0.504672,0.504672,0.503457,0.504672,0.501427,0.505076,0.503457,0.504672,0.490314,0.503457,0.504672,0.388088,0.274048,0.443087
min,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,12.25000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000
50%,24.50000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,36.75000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000
max,49.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [44]:
df_preprocess.select_dtypes(include=['number']).columns

Index(['ehr', 'fiebre', 'pustulas', 'distribucion_pustulas',
       'sobre_areas_enrojecidas', 'otros_sintomas', 'dolor_articulaciones',
       'lesiones_escamosas', 'psoriasis', 'familiares_psoriasis',
       'brotes_similares', 'lesion_mas_de_3_meses', 'sintomas_post_infeccion',
       'cambio_medicamento', 'estres_infecciones', 'vacunacion_reciente',
       'resultado_No', 'resultado_Probablemente', 'resultado_Sospecha'],
      dtype='object')

In [55]:
df_clasify = training(df_preprocess)
df_clasify.head()

,ehr,fiebre,pustulas,distribucion_pustulas,sobre_areas_enrojecidas,otros_sintomas,dolor_articulaciones,lesiones_escamosas,psoriasis,familiares_psoriasis,...,lesion_mas_de_3_meses,sintomas_post_infeccion,cambio_medicamento,estres_infecciones,vacunacion_reciente,fecha,resultado_No,resultado_Probablemente,resultado_Sospecha,cluster
0,0,1,1,0,0,0,0,1,0,0,...,1,0,0,1,0,2023-03-16 22:25:48,0,0,1,2
1,1,1,0,0,0,1,1,0,1,0,...,1,1,1,0,0,2024-03-10 13:17:14,0,0,1,2
2,2,0,1,0,0,0,1,0,1,1,...,0,1,1,0,1,2023-01-11 10:41:45,0,0,1,2
3,3,1,1,0,0,1,0,0,1,1,...,0,0,0,0,0,2023-04-26 10:16:23,0,0,1,2
4,4,1,0,0,0,0,1,1,1,0,...,1,1,1,1,1,2020-06-25 15:25:10,0,0,1,8


In [54]:
get_patients_from_same_cluster(3, df_clasify)

,ehr,fiebre,pustulas,distribucion_pustulas,sobre_areas_enrojecidas,otros_sintomas,dolor_articulaciones,lesiones_escamosas,psoriasis,familiares_psoriasis,...,lesion_mas_de_3_meses,sintomas_post_infeccion,cambio_medicamento,estres_infecciones,vacunacion_reciente,fecha,resultado_No,resultado_Probablemente,resultado_Sospecha,cluster
0,0,1,1,0,0,0,0,1,0,0,...,1,0,0,1,0,2023-03-16 22:25:48,0,0,1,2
1,1,1,0,0,0,1,1,0,1,0,...,1,1,1,0,0,2024-03-10 13:17:14,0,0,1,2
2,2,0,1,0,0,0,1,0,1,1,...,0,1,1,0,1,2023-01-11 10:41:45,0,0,1,2
3,3,1,1,0,0,1,0,0,1,1,...,0,0,0,0,0,2023-04-26 10:16:23,0,0,1,2
